In [1]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import pandas as pd
df1 = pd.read_csv(r"D:\desktop\B\数据\标准化后数据\df1.csv")
df2 = pd.read_excel(r"D:\desktop\B\数据\标准化后数据\df2.xlsx")
df1[['mort', 'CONS','MV','CRRT']] = df1[['mort',  'CONS','MV','CRRT']].astype('category')
df2[['mort',  'CONS','MV','CRRT']] = df2[['mort',  'CONS','MV','CRRT']].astype('category')
selected_features = ["CONS", "LDH", "MV", "AST", "CRRT", "U", "L", "HR", "D.Dimer", "CR", "age", "ALT"]
target = 'mort'
X = df1[selected_features]
y = df1[target]
X_train, X_internal_test, y_train, y_internal_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_external_test = df2[selected_features]
y_external_test = df2[target]

In [4]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.neighbors import KNeighborsClassifier

RANDOM_SEED = 42

# 定义计算置信区间的函数
def bootstrap_metric_ci(y_true, y_pred, metric_func, n_bootstraps=10000, ci=95, random_state=42):
    rng = np.random.RandomState(random_state)
    bootstrapped_scores = []
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    for _ in range(n_bootstraps):
        indices = resample(np.arange(len(y_true)), replace=True, random_state=rng)
        score = metric_func(y_true[indices], y_pred[indices])
        bootstrapped_scores.append(score)
    
    lower_bound = np.percentile(bootstrapped_scores, (100-ci)/2)
    upper_bound = np.percentile(bootstrapped_scores, 100-(100-ci)/2)
    return lower_bound, upper_bound

# 定义KNN模型参数
params_knn = {
    'n_neighbors': [3, 5, 10, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

best_models_knn = {}

# 逐步添加特征并训练模型
for i in range(1, len(selected_features) + 1):
    current_features = selected_features[:i]
    X_train_current = X_train[current_features]
    X_internal_test_current = X_internal_test[current_features]
    X_external_test_current = X_external_test[current_features]

    model_knn = KNeighborsClassifier()
    grid_knn = GridSearchCV(model_knn, params_knn, cv=5, scoring='roc_auc', n_jobs=-1)
    grid_knn.fit(X_train_current, y_train)

    best_model_knn = grid_knn.best_estimator_

    # 使用交叉验证在训练集和内部验证集上进行预测
    y_train_pred_cv = cross_val_predict(best_model_knn, X_train_current, y_train, cv=10, method='predict')
    y_train_proba_cv = cross_val_predict(best_model_knn, X_train_current, y_train, cv=10, method='predict_proba')[:, 1]
    
    y_internal_pred_cv = cross_val_predict(best_model_knn, X_internal_test_current, y_internal_test, cv=10, method='predict')
    y_internal_proba_cv = cross_val_predict(best_model_knn, X_internal_test_current, y_internal_test, cv=10, method='predict_proba')[:, 1]

    # 进行外部测试集预测
    y_external_pred_knn = best_model_knn.predict(X_external_test_current)
    y_external_proba_knn = best_model_knn.predict_proba(X_external_test_current)[:, 1]

    # 计算训练集的各项指标及其置信区间
    train_metrics = {
        'accuracy': accuracy_score(y_train, y_train_pred_cv),
        'precision': precision_score(y_train, y_train_pred_cv, average='weighted', zero_division=0),
        'recall': recall_score(y_train, y_train_pred_cv, average='weighted'),
        'f1_score': f1_score(y_train, y_train_pred_cv, average='weighted'),
        'auc': roc_auc_score(y_train, y_train_proba_cv)
    }

    train_ci = {
        'accuracy_ci': bootstrap_metric_ci(y_train, y_train_pred_cv, accuracy_score),
        'precision_ci': bootstrap_metric_ci(y_train, y_train_pred_cv, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
        'recall_ci': bootstrap_metric_ci(y_train, y_train_pred_cv, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
        'f1_score_ci': bootstrap_metric_ci(y_train, y_train_pred_cv, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
        'auc_ci': bootstrap_metric_ci(y_train, y_train_proba_cv, roc_auc_score)
    }

    # 计算内部验证集的各项指标及其置信区间
    internal_metrics = {
        'accuracy': accuracy_score(y_internal_test, y_internal_pred_cv),
        'precision': precision_score(y_internal_test, y_internal_pred_cv, average='weighted', zero_division=0),
        'recall': recall_score(y_internal_test, y_internal_pred_cv, average='weighted'),
        'f1_score': f1_score(y_internal_test, y_internal_pred_cv, average='weighted'),
        'auc': roc_auc_score(y_internal_test, y_internal_proba_cv)
    }

    internal_ci = {
        'accuracy_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_cv, accuracy_score),
        'precision_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_cv, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
        'recall_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_cv, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
        'f1_score_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_cv, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
        'auc_ci': bootstrap_metric_ci(y_internal_test, y_internal_proba_cv, roc_auc_score)
    }

    # 计算外部验证集的各项指标及其置信区间
    external_metrics = {
        'accuracy': accuracy_score(y_external_test, y_external_pred_knn),
        'precision': precision_score(y_external_test, y_external_pred_knn, average='weighted', zero_division=0),
        'recall': recall_score(y_external_test, y_external_pred_knn, average='weighted'),
        'f1_score': f1_score(y_external_test, y_external_pred_knn, average='weighted'),
        'auc': roc_auc_score(y_external_test, y_external_proba_knn)
    }

    external_ci = {
        'accuracy_ci': bootstrap_metric_ci(y_external_test, y_external_pred_knn, accuracy_score),
        'precision_ci': bootstrap_metric_ci(y_external_test, y_external_pred_knn, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
        'recall_ci': bootstrap_metric_ci(y_external_test, y_external_pred_knn, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
        'f1_score_ci': bootstrap_metric_ci(y_external_test, y_external_pred_knn, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
        'auc_ci': bootstrap_metric_ci(y_external_test, y_external_proba_knn, roc_auc_score)
    }

    best_models_knn[i] = {
        'features': current_features,
        'train_metrics': train_metrics,
        'train_ci': train_ci,
        'internal_metrics': internal_metrics,
        'internal_ci': internal_ci,
        'external_metrics': external_metrics,
        'external_ci': external_ci,
        'best_params': grid_knn.best_params_
    }

# 输出每个模型的结果
for i, results in best_models_knn.items():
    print(f"\nModel with top {i} features: {results['features']}")
    print("Best Parameters:", results['best_params'])
    
    print("Training Set Metrics:")
    for metric, value in results['train_metrics'].items():
        print(f"{metric.capitalize()}: {value:.4f} 置信区间: {results['train_ci'][metric+'_ci'][0]:.4f}-{results['train_ci'][metric+'_ci'][1]:.4f}")

    print("Internal Validation Metrics:")
    for metric, value in results['internal_metrics'].items():
        print(f"{metric.capitalize()}: {value:.4f} 置信区间: {results['internal_ci'][metric+'_ci'][0]:.4f}-{results['internal_ci'][metric+'_ci'][1]:.4f}")
    
    print("External Validation Metrics:")
    for metric, value in results['external_metrics'].items():
        print(f"{metric.capitalize()}: {value:.4f} 置信区间: {results['external_ci'][metric+'_ci'][0]:.4f}-{results['external_ci'][metric+'_ci'][1]:.4f}")



Model with top 1 features: ['CONS']
Best Parameters: {'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'uniform'}
Training Set Metrics:
Accuracy: 0.7647 置信区间: 0.7059-0.8235
Precision: 0.7107 置信区间: 0.6031-0.8124
Recall: 0.7647 置信区间: 0.7059-0.8235
F1_score: 0.6940 置信区间: 0.6131-0.7706
Auc: 0.8368 置信区间: 0.7701-0.8986
Internal Validation Metrics:
Accuracy: 0.7273 置信区间: 0.6364-0.8182
Precision: 0.7346 置信区间: 0.6415-0.8332
Recall: 0.7273 置信区间: 0.6364-0.8182
F1_score: 0.7306 置信区间: 0.6376-0.8212
Auc: 0.7497 置信区间: 0.6384-0.8512
External Validation Metrics:
Accuracy: 0.7500 置信区间: 0.6635-0.8269
Precision: 0.5625 置信区间: 0.4402-0.6838
Recall: 0.7500 置信区间: 0.6635-0.8269
F1_score: 0.6429 置信区间: 0.5292-0.7486
Auc: 0.7628 置信区间: 0.6643-0.8610

Model with top 2 features: ['CONS', 'LDH']
Best Parameters: {'metric': 'manhattan', 'n_neighbors': 15, 'weights': 'distance'}
Training Set Metrics:
Accuracy: 0.8676 置信区间: 0.8186-0.9118
Precision: 0.8651 置信区间: 0.8161-0.9124
Recall: 0.8676 置信区间: 0.8186-0.9118
F1_sco

In [5]:
import pandas as pd
import os

# 设置输出文件夹路径
output_folder = r"D:\desktop\B\结果\表格\内部外部验证结果"
os.makedirs(output_folder, exist_ok=True)  # 确保输出文件夹存在

# 用于存储结果的列表
output_data = []

# 遍历模型结果并将其组织为字典
for i, results in best_models_knn.items():
    model_name = f"Model with top {i} features"
    for metric in results['train_metrics'].keys():
        output_data.append({
            'Model': model_name,
            'Dataset': 'Training Set',
            'Metric': metric.capitalize(),
            'Value': results['train_metrics'][metric],
            'Confidence Interval': f"{results['train_ci'][metric+'_ci'][0]:.4f}-{results['train_ci'][metric+'_ci'][1]:.4f}"
        })
        output_data.append({
            'Model': model_name,
            'Dataset': 'Internal Validation Set',
            'Metric': metric.capitalize(),
            'Value': results['internal_metrics'][metric],
            'Confidence Interval': f"{results['internal_ci'][metric+'_ci'][0]:.4f}-{results['internal_ci'][metric+'_ci'][1]:.4f}"
        })
        output_data.append({
            'Model': model_name,
            'Dataset': 'External Validation Set',
            'Metric': metric.capitalize(),
            'Value': results['external_metrics'][metric],
            'Confidence Interval': f"{results['external_ci'][metric+'_ci'][0]:.4f}-{results['external_ci'][metric+'_ci'][1]:.4f}"
        })

# 将列表转换为 DataFrame
output_df = pd.DataFrame(output_data)

# 设置文件名
output_file = os.path.join(output_folder, "KNN_Results.xlsx")

# 将 DataFrame 保存为 Excel 文件
output_df.to_excel(output_file, index=False)

print(f"结果已保存至：{output_file}")


结果已保存至：D:\desktop\B\结果\表格\内部外部验证结果\KNN_Results.xlsx


In [4]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.utils import resample
from sklearn.model_selection import cross_val_predict
from sklearn.neighbors import KNeighborsClassifier

RANDOM_SEED = 42

# 定义计算置信区间的函数
def bootstrap_metric_ci(y_true, y_pred, metric_func, n_bootstraps=10000, ci=95, random_state=42):
    rng = np.random.RandomState(random_state)
    bootstrapped_scores = []
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    for _ in range(n_bootstraps):
        indices = resample(np.arange(len(y_true)), replace=True, random_state=rng)
        score = metric_func(y_true[indices], y_pred[indices])
        bootstrapped_scores.append(score)
    
    lower_bound = np.percentile(bootstrapped_scores, (100-ci)/2)
    upper_bound = np.percentile(bootstrapped_scores, 100-(100-ci)/2)
    return lower_bound, upper_bound

# 使用最佳参数创建KNN模型
model_knn_best = KNeighborsClassifier(metric='manhattan', n_neighbors=15, weights='distance')

# 训练模型
model_knn_best.fit(X_train[selected_features], y_train)

# 进行训练集预测
y_train_pred_knn = cross_val_predict(model_knn_best, X_train[selected_features], y_train, cv=10, method='predict')
y_train_proba_knn = cross_val_predict(model_knn_best, X_train[selected_features], y_train, cv=10, method='predict_proba')[:, 1]

# 使用交叉验证在内部验证集上进行预测
y_internal_pred_knn = cross_val_predict(model_knn_best, X_internal_test[selected_features], y_internal_test, cv=10, method='predict')
y_internal_proba_knn = cross_val_predict(model_knn_best, X_internal_test[selected_features], y_internal_test, cv=10, method='predict_proba')[:, 1]

# 对外部验证集进行单次预测（这里因为没有交叉验证，所以直接预测）
y_external_pred_knn = model_knn_best.predict(X_external_test[selected_features])
y_external_proba_knn = model_knn_best.predict_proba(X_external_test[selected_features])[:, 1]

# 计算训练集的各项指标及其置信区间
train_metrics = {
    'accuracy': accuracy_score(y_train, y_train_pred_knn),
    'precision': precision_score(y_train, y_train_pred_knn, average='weighted', zero_division=0),
    'recall': recall_score(y_train, y_train_pred_knn, average='weighted'),
    'f1_score': f1_score(y_train, y_train_pred_knn, average='weighted'),
    'auc': roc_auc_score(y_train, y_train_proba_knn)
}

train_ci = {
    'accuracy_ci': bootstrap_metric_ci(y_train, y_train_pred_knn, accuracy_score),
    'precision_ci': bootstrap_metric_ci(y_train, y_train_pred_knn, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
    'recall_ci': bootstrap_metric_ci(y_train, y_train_pred_knn, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
    'f1_score_ci': bootstrap_metric_ci(y_train, y_train_pred_knn, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
    'auc_ci': bootstrap_metric_ci(y_train, y_train_proba_knn, roc_auc_score)
}

# 计算内部验证集的各项指标及其置信区间
internal_metrics = {
    'accuracy': accuracy_score(y_internal_test, y_internal_pred_knn),
    'precision': precision_score(y_internal_test, y_internal_pred_knn, average='weighted', zero_division=0),
    'recall': recall_score(y_internal_test, y_internal_pred_knn, average='weighted'),
    'f1_score': f1_score(y_internal_test, y_internal_pred_knn, average='weighted'),
    'auc': roc_auc_score(y_internal_test, y_internal_proba_knn)
}

internal_ci = {
    'accuracy_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_knn, accuracy_score),
    'precision_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_knn, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
    'recall_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_knn, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
    'f1_score_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_knn, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
    'auc_ci': bootstrap_metric_ci(y_internal_test, y_internal_proba_knn, roc_auc_score)
}

# 计算外部验证集的各项指标及其置信区间
external_metrics = {
    'accuracy': accuracy_score(y_external_test, y_external_pred_knn),
    'precision': precision_score(y_external_test, y_external_pred_knn, average='weighted', zero_division=0),
    'recall': recall_score(y_external_test, y_external_pred_knn, average='weighted'),
    'f1_score': f1_score(y_external_test, y_external_pred_knn, average='weighted'),
    'auc': roc_auc_score(y_external_test, y_external_proba_knn)
}

external_ci = {
    'accuracy_ci': bootstrap_metric_ci(y_external_test, y_external_pred_knn, accuracy_score),
    'precision_ci': bootstrap_metric_ci(y_external_test, y_external_pred_knn, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
    'recall_ci': bootstrap_metric_ci(y_external_test, y_external_pred_knn, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
    'f1_score_ci': bootstrap_metric_ci(y_external_test, y_external_pred_knn, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
    'auc_ci': bootstrap_metric_ci(y_external_test, y_external_proba_knn, roc_auc_score)
}

# 输出结果
print(f"Model with top {len(selected_features)} features: {selected_features}")
print("Best Parameters: {'metric': 'manhattan', 'n_neighbors': 15, 'weights': 'distance'}")

print("Training Set Metrics:")
for metric, value in train_metrics.items():
    print(f"{metric.capitalize()}: {value:.4f} 置信区间: {train_ci[metric+'_ci'][0]:.4f}-{train_ci[metric+'_ci'][1]:.4f}")

print("Internal Validation Set Metrics:")
for metric, value in internal_metrics.items():
    print(f"{metric.capitalize()}: {value:.4f} 置信区间: {internal_ci[metric+'_ci'][0]:.4f}-{internal_ci[metric+'_ci'][1]:.4f}")
    
print("External Validation Set Metrics:")
for metric, value in external_metrics.items():
    print(f"{metric.capitalize()}: {value:.4f} 置信区间: {external_ci[metric+'_ci'][0]:.4f}-{external_ci[metric+'_ci'][1]:.4f}")


Model with top 12 features: ['CONS', 'LDH', 'MV', 'AST', 'CRRT', 'U', 'L', 'HR', 'D.Dimer', 'CR', 'age', 'ALT']
Best Parameters: {'metric': 'manhattan', 'n_neighbors': 15, 'weights': 'distance'}
Training Set Metrics:
Accuracy: 0.8922 置信区间: 0.8480-0.9314
Precision: 0.8935 置信区间: 0.8502-0.9339
Recall: 0.8922 置信区间: 0.8480-0.9314
F1_score: 0.8840 置信区间: 0.8327-0.9283
Auc: 0.9316 置信区间: 0.8855-0.9690
Internal Validation Set Metrics:
Accuracy: 0.7727 置信区间: 0.6818-0.8523
Precision: 0.7597 置信区间: 0.6515-0.8616
Recall: 0.7727 置信区间: 0.6818-0.8523
F1_score: 0.7371 置信区间: 0.6230-0.8397
Auc: 0.9219 置信区间: 0.8588-0.9718
External Validation Set Metrics:
Accuracy: 0.8654 置信区间: 0.7981-0.9327
Precision: 0.8738 置信区间: 0.8082-0.9305
Recall: 0.8654 置信区间: 0.7981-0.9327
F1_score: 0.8500 置信区间: 0.7634-0.9246
Auc: 0.8767 置信区间: 0.7752-0.9629


In [6]:
import pandas as pd

# 获取训练集的真实标签和预测概率
y_train_true = y_train.values
train_data = pd.DataFrame({
    'True_Label': y_train_true,
    'Predicted_Probability': y_train_proba_knn
})
train_data.to_csv(r'D:\desktop\B\结果\表格\KNN真实标签和预测概率\train_labels_and_probabilities.csv', index=False)

# 获取内部验证集的真实标签和预测概率
y_internal_true = y_internal_test.values
internal_data = pd.DataFrame({
    'True_Label': y_internal_true,
    'Predicted_Probability': y_internal_proba_knn
})
internal_data.to_csv(r'D:\desktop\B\结果\表格\KNN真实标签和预测概率\internal_validation_labels_and_probabilities.csv', index=False)

# 获取外部验证集的真实标签和预测概率
y_external_true = y_external_test.values
external_data = pd.DataFrame({
    'True_Label': y_external_true,
    'Predicted_Probability': y_external_proba_knn
})
external_data.to_csv(r'D:\desktop\B\结果\表格\KNN真实标签和预测概率\external_validation_labels_and_probabilities.csv', index=False)

print("训练集、内部验证集和外部验证集的标签和预测概率已保存到指定文件夹。")


训练集、内部验证集和外部验证集的标签和预测概率已保存到指定文件夹。


In [2]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.utils import resample
from sklearn.model_selection import cross_val_predict
from sklearn.neighbors import KNeighborsClassifier

RANDOM_SEED = 42

# 定义计算置信区间的函数
def bootstrap_metric_ci(y_true, y_pred, metric_func, n_bootstraps=10000, ci=95, random_state=42):
    rng = np.random.RandomState(random_state)
    bootstrapped_scores = []
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    for _ in range(n_bootstraps):
        indices = resample(np.arange(len(y_true)), replace=True, random_state=rng)
        score = metric_func(y_true[indices], y_pred[indices])
        bootstrapped_scores.append(score)
    
    lower_bound = np.percentile(bootstrapped_scores, (100-ci)/2)
    upper_bound = np.percentile(bootstrapped_scores, 100-(100-ci)/2)
    return lower_bound, upper_bound
selected_features = ['CONS', 'LDH', 'MV', 'AST', 'CRRT', 'U', 'L']
# 使用最佳参数创建KNN模型
model_knn_best = KNeighborsClassifier(metric='manhattan', n_neighbors=15, weights='distance')

# 训练模型
model_knn_best.fit(X_train[selected_features], y_train)

# 进行训练集预测
y_train_pred_knn = cross_val_predict(model_knn_best, X_train[selected_features], y_train, cv=10, method='predict')
y_train_proba_knn = cross_val_predict(model_knn_best, X_train[selected_features], y_train, cv=10, method='predict_proba')[:, 1]

# 使用交叉验证在内部验证集上进行预测
y_internal_pred_knn = cross_val_predict(model_knn_best, X_internal_test[selected_features], y_internal_test, cv=10, method='predict')
y_internal_proba_knn = cross_val_predict(model_knn_best, X_internal_test[selected_features], y_internal_test, cv=10, method='predict_proba')[:, 1]

# 对外部验证集进行单次预测（这里因为没有交叉验证，所以直接预测）
y_external_pred_knn = model_knn_best.predict(X_external_test[selected_features])
y_external_proba_knn = model_knn_best.predict_proba(X_external_test[selected_features])[:, 1]

# 计算训练集的各项指标及其置信区间
train_metrics = {
    'accuracy': accuracy_score(y_train, y_train_pred_knn),
    'precision': precision_score(y_train, y_train_pred_knn, average='weighted', zero_division=0),
    'recall': recall_score(y_train, y_train_pred_knn, average='weighted'),
    'f1_score': f1_score(y_train, y_train_pred_knn, average='weighted'),
    'auc': roc_auc_score(y_train, y_train_proba_knn)
}

train_ci = {
    'accuracy_ci': bootstrap_metric_ci(y_train, y_train_pred_knn, accuracy_score),
    'precision_ci': bootstrap_metric_ci(y_train, y_train_pred_knn, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
    'recall_ci': bootstrap_metric_ci(y_train, y_train_pred_knn, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
    'f1_score_ci': bootstrap_metric_ci(y_train, y_train_pred_knn, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
    'auc_ci': bootstrap_metric_ci(y_train, y_train_proba_knn, roc_auc_score)
}

# 计算内部验证集的各项指标及其置信区间
internal_metrics = {
    'accuracy': accuracy_score(y_internal_test, y_internal_pred_knn),
    'precision': precision_score(y_internal_test, y_internal_pred_knn, average='weighted', zero_division=0),
    'recall': recall_score(y_internal_test, y_internal_pred_knn, average='weighted'),
    'f1_score': f1_score(y_internal_test, y_internal_pred_knn, average='weighted'),
    'auc': roc_auc_score(y_internal_test, y_internal_proba_knn)
}

internal_ci = {
    'accuracy_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_knn, accuracy_score),
    'precision_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_knn, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
    'recall_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_knn, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
    'f1_score_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_knn, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
    'auc_ci': bootstrap_metric_ci(y_internal_test, y_internal_proba_knn, roc_auc_score)
}

# 计算外部验证集的各项指标及其置信区间
external_metrics = {
    'accuracy': accuracy_score(y_external_test, y_external_pred_knn),
    'precision': precision_score(y_external_test, y_external_pred_knn, average='weighted', zero_division=0),
    'recall': recall_score(y_external_test, y_external_pred_knn, average='weighted'),
    'f1_score': f1_score(y_external_test, y_external_pred_knn, average='weighted'),
    'auc': roc_auc_score(y_external_test, y_external_proba_knn)
}

external_ci = {
    'accuracy_ci': bootstrap_metric_ci(y_external_test, y_external_pred_knn, accuracy_score),
    'precision_ci': bootstrap_metric_ci(y_external_test, y_external_pred_knn, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
    'recall_ci': bootstrap_metric_ci(y_external_test, y_external_pred_knn, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
    'f1_score_ci': bootstrap_metric_ci(y_external_test, y_external_pred_knn, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
    'auc_ci': bootstrap_metric_ci(y_external_test, y_external_proba_knn, roc_auc_score)
}

# 输出结果
print(f"Model with top {len(selected_features)} features: {selected_features}")
print("Best Parameters: {'metric': 'manhattan', 'n_neighbors': 15, 'weights': 'distance'}")

print("Training Set Metrics:")
for metric, value in train_metrics.items():
    print(f"{metric.capitalize()}: {value:.4f} 置信区间: {train_ci[metric+'_ci'][0]:.4f}-{train_ci[metric+'_ci'][1]:.4f}")

print("Internal Validation Set Metrics:")
for metric, value in internal_metrics.items():
    print(f"{metric.capitalize()}: {value:.4f} 置信区间: {internal_ci[metric+'_ci'][0]:.4f}-{internal_ci[metric+'_ci'][1]:.4f}")
    
print("External Validation Set Metrics:")
for metric, value in external_metrics.items():
    print(f"{metric.capitalize()}: {value:.4f} 置信区间: {external_ci[metric+'_ci'][0]:.4f}-{external_ci[metric+'_ci'][1]:.4f}")


Model with top 7 features: ['CONS', 'LDH', 'MV', 'AST', 'CRRT', 'U', 'L']
Best Parameters: {'metric': 'manhattan', 'n_neighbors': 15, 'weights': 'distance'}
Training Set Metrics:
Accuracy: 0.8824 置信区间: 0.8333-0.9265
Precision: 0.8783 置信区间: 0.8298-0.9244
Recall: 0.8824 置信区间: 0.8333-0.9265
F1_score: 0.8773 置信区间: 0.8263-0.9222
Auc: 0.9187 置信区间: 0.8655-0.9619
Internal Validation Set Metrics:
Accuracy: 0.8409 置信区间: 0.7614-0.9091
Precision: 0.8378 置信区间: 0.7552-0.9181
Recall: 0.8409 置信区间: 0.7614-0.9091
F1_score: 0.8297 置信区间: 0.7373-0.9076
Auc: 0.9046 置信区间: 0.8186-0.9701
External Validation Set Metrics:
Accuracy: 0.8942 置信区间: 0.8365-0.9519
Precision: 0.9073 置信区间: 0.8655-0.9546
Recall: 0.8942 置信区间: 0.8365-0.9519
F1_score: 0.8835 置信区间: 0.8071-0.9490
Auc: 0.8750 置信区间: 0.7788-0.9597


In [4]:
import pandas as pd

# 获取训练集的真实标签和预测概率
y_train_true = y_train.values
train_data = pd.DataFrame({
    'True_Label': y_train_true,
    'Predicted_Probability': y_train_proba_knn
})
train_data.to_csv(r'D:\desktop\B\结果\表格\KNN真实标签和预测概率(7)\train_labels_and_probabilities.csv', index=False)

# 获取内部验证集的真实标签和预测概率
y_internal_true = y_internal_test.values
internal_data = pd.DataFrame({
    'True_Label': y_internal_true,
    'Predicted_Probability': y_internal_proba_knn
})
internal_data.to_csv(r'D:\desktop\B\结果\表格\KNN真实标签和预测概率(7)\internal_validation_labels_and_probabilities.csv', index=False)

# 获取外部验证集的真实标签和预测概率
y_external_true = y_external_test.values
external_data = pd.DataFrame({
    'True_Label': y_external_true,
    'Predicted_Probability': y_external_proba_knn
})
external_data.to_csv(r'D:\desktop\B\结果\表格\KNN真实标签和预测概率(7)\external_validation_labels_and_probabilities.csv', index=False)

print("训练集、内部验证集和外部验证集的标签和预测概率已保存到指定文件夹。")


训练集、内部验证集和外部验证集的标签和预测概率已保存到指定文件夹。
